<a href="https://colab.research.google.com/github/neuralsrg/EDA/blob/main/retrieve_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieving data from a database

## 1. SQL


In [ ]:
! wget -P data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/classic_rock.db

In [ ]:
import sqlite3 as sq3
import pandas as pd
import pandas.io.sql as pds

path = 'data/classic_rock.db'
con = sq3.Connection(path)

# query
query = ''' SELECT * FROM rock_songs;
'''
# table to pandas
data = pds.read_sql(query, con)
data.head(10)

,Song,Artist,Release_Year,PlayCount
0,Caught Up in You,.38 Special,1982.0,82
1,Hold On Loosely,.38 Special,1981.0,85
2,Rockin' Into the Night,.38 Special,1980.0,18
3,Art For Arts Sake,10cc,1975.0,1
4,Kryptonite,3 Doors Down,2000.0,13
5,Loser,3 Doors Down,2000.0,1
6,When I'm Gone,3 Doors Down,2002.0,6
7,What's Up?,4 Non Blondes,1992.0,3
8,Take On Me,a-ha,1985.0,1
9,Back In Black,AC/DC,1980.0,97


In [ ]:
query = '''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''
data_generator = pds.read_sql(
    query,
    con,
    coerce_float=True, # whether to force float cast
    parse_dates=['Release_Year'], # which columns to interpret as dates
    chunksize=5
)
                   
for i, data in enumerate(data_generator):
  if i == 4:
    break
  print(f'{i}-th data chunks')
  display(data)

0-th data chunks


,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1970-01-01 00:32:47,23,6.565217
1,Led Zeppelin,1970-01-01 00:32:49,18,21.000000
2,The Beatles,1970-01-01 00:32:45,15,3.800000
3,The Beatles,1970-01-01 00:32:48,13,13.000000
4,The Beatles,1970-01-01 00:32:49,13,15.000000


1-th data chunks


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:50,12,13.166667
1,Led Zeppelin,1970-01-01 00:32:55,12,14.166667
2,Pink Floyd,1970-01-01 00:32:59,11,41.454545
3,Pink Floyd,1970-01-01 00:32:53,10,29.100000
4,The Doors,1970-01-01 00:32:47,10,28.900000


2-th data chunks


,Artist,Release_Year,num_songs,avg_plays
0,Fleetwood Mac,1970-01-01 00:32:57,9,35.666667
1,Jimi Hendrix,1970-01-01 00:32:47,9,24.888889
2,The Beatles,1970-01-01 00:32:43,9,2.444444
3,The Beatles,1970-01-01 00:32:44,9,3.111111
4,Elton John,1970-01-01 00:32:53,8,18.500000


3-th data chunks


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:51,8,47.750000
1,Led Zeppelin,1970-01-01 00:32:53,8,34.125000
2,Boston,1970-01-01 00:32:56,7,69.285714
3,Rolling Stones,1970-01-01 00:32:49,7,36.142857
4,Van Halen,1970-01-01 00:32:58,7,51.142857


## 2. NoSQL

In [ ]:
from pymongo import MongoClient

con = MongoClient()
databases = con.list_database_names # see database n  ames 
db = con.database_name # retrieve a certain database 

# cursor object 
cursor = db.collection_name.find(query) # python iterator 

# dataframe 
df = pd.DataFrame(list(cursor))

## 3. Cloud data access

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
data = pd.read_csv(url, header=None)

data.head(4)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
